# 00000: Model prediction and tiling for validation
Authors: Tobias G. Mueller, Mark A. Buckner
Last modified: 4 Dec 2024
Contact: __________

**Summary**: Here, we predict on an orthomosaic using out pretrained model. 
We then split the predictions and image into smaller tiles and take 20% to
be ground truthed and used for model validation.


This script outputs 
- model predictoins in yolov5 format 
- a random 20% of tiles into a testset folder 

The data used in this script was generated in:
    `AIggregation/notebooks/01_preprocessing.ipynb`

Following this script the tiles in the target folder need to be annotated in label studio to create ground truth detections
Before `03_validation_and_optimization.ipynb` can be run

In [1]:
#imports 
import os
import fiftyone as fo
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from PIL import Image 


# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())




cwd is /home/tmueller/github/AIggregation/notebooks
cwd changed to /home/tmueller/github/AIggregation


In [ ]:


# set paths
# -------------------------------------------------------------------------------------------------------------------- #
image_directory = "datasets/drone_ortho/ortho_clip_23april.png"
model_path = "AIggregation_yolov5m/weights/best.pt"
export_directory = "datasets/export_predictions/temp"
# -------------------------------------------------------------------------------------------------------------------- #





# Import ortho image as a fiftyone dataset 
dataset_full = fo.Dataset.from_images(
    [image_directory]
)

# specify AI detection model to use for predictions
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=model_path, #specify path to trained model
    confidence_threshold=0.25,
    device="cpu", # "cpu" or 'cuda' for GPU
)

# function for sliced predictions from sahi
def predict_with_slicing(sample, label_field, **kwargs):
    result = get_sliced_prediction(
        sample.filepath, detection_model, verbose=0, **kwargs
    )
    sample[label_field] = fo.Detections(detections=result.to_fiftyone_detections())

# slice at training image size
for sample in dataset_full.iter_samples(progress=True, autosave=True):
    predict_with_slicing(sample,
                         label_field="prediction",
                         slice_height=608, 
                         slice_width=608,
                         overlap_height_ratio = .4, 
                         overlap_width_ratio=.4
    )


#launch fiftyone session to see predictions
session = fo.launch_app(dataset_full)


#export predictions
dataset_full.export(
        export_dir=export_directory,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="prediction",
        include_confidence=True
    )

# NEW NUMBERS BELOW FOR SINLGE PREDICITON ARE NEEDED

# took 7m 53s for loading image, prediction, and export with CPU (AMD Ryzen 5 5500 3.6 GHz 6-Core Processor)
# took 2m 38s on GPU (EVGA SC GAMING GeForce GTX 1060 3GB 3 GB Video Card)
# 64 gb DDR4-3200 ram


 100% |█████████████████████| 1/1 [22.5ms elapsed, 0s remaining, 54.6 samples/s] 


12/04/2024 09:09:29 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [22.5ms elapsed, 0s remaining, 54.6 samples/s] 


 100% |█████████████████████| 1/1 [7.8m elapsed, 0s remaining, 0.0 samples/s] 


12/04/2024 09:17:16 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [7.8m elapsed, 0s remaining, 0.0 samples/s] 


 100% |█████████████████████| 1/1 [791.6ms elapsed, 0s remaining, 1.3 samples/s] 


12/04/2024 09:17:20 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [791.6ms elapsed, 0s remaining, 1.3 samples/s] 


In [ ]:

''' 
To compare the predicitons vs a labeled test set, we tile the exports and ground truth a random 20% of the tiles.

The below script takes a folder with yolov5 formated images and labels, tiles them into specified size tiles and saves a percent of random tiles into a new folder. 


'''

# first set max image pixels to none
# otherwise pillow thinks its a bomb DOS attack 
Image.MAX_IMAGE_PIXELS = None


# run yolo_tile script for each prediction
%run scripts/yolo_tile_modified.py -source ./datasets/export_predictions -target ./datasets/testset/tiled_testset -ext .png  -size 608 -ratio 0.2


'''
    PARAMETERS:

    -source         Source folder with images and labels needed to be tiled. 
    -target         Target folder for a new sliced dataset. Default: 
    -ext            Image extension in a dataset. Default: .JPG
    -size           Size of a tile. Default: 608
    -ratio          test split ratio. Default: 0.2 (i.e. 20% of tiles in the test set)
'''

# the test subset images can now be annotated in labelstudio for groundtruth labels

